LIMPIAR AMBOS CSV

In [1]:
import sys
from pathlib import Path
import pandas as pd

# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent/"src"
sys.path.append(str(path_inicio))
from utilidades.constantes import DATA_PATH, DATA_PROCESADA_PATH, PATRON_MOSTRADOR_PATH, MOSTRADOR_SALIDA_PATH, PATRON_FACTURACION_PATH, FACTURACION_SALIDA_PATH

LIMPIO MOSTRADOR

In [2]:

# 1. Leer archivo consolidado
df = pd.read_csv(MOSTRADOR_SALIDA_PATH, encoding='utf-8-sig')

# 2. Eliminar ventas anuladas (si existe la columna "Anulado")
if "Anulado" in df.columns:
    df = df[df["Anulado"].str.lower() == "no"]
else:
    print("No se encontró columna 'Anulado'.")

# 3. Estandarizar columna de fecha
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

# 4. Asegurar que TotalVenta y Cantidad sean numéricos
df['TotalVenta'] = pd.to_numeric(df['TotalVenta'].round(0).astype('Int64') , errors='coerce')
if 'Cantidad' in df.columns:
    df['Cantidad'] = pd.to_numeric(df['Cantidad'], errors='coerce')


columnas_utiles = ['Fecha', 'Producto', 'Cantidad', 'TotalVenta','Cond.Pago','Cliente']
df = df[columnas_utiles]

# 6. Guardar CSV limpio
df.to_csv(MOSTRADOR_SALIDA_PATH, index=False, encoding='utf-8-sig')
print("Archivo limpio guardado en data_procesada/ventas_limpio.csv")

# Verificá primeras filas
df.head()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33400\330635690.py:2: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(MOSTRADOR_SALIDA_PATH, encoding='utf-8-sig')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_33400\330635690.py:11: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')


Archivo limpio guardado en data_procesada/ventas_limpio.csv


,Fecha,Producto,Cantidad,TotalVenta,Cond.Pago,Cliente
0,2023-03-30 17:13:01,ENVIO A DOMICILIO,1.0,63500,.CREDITO,CASINELLI MARIO OSCAR
1,2023-03-30 11:14:11,BEBIDA GASEOSA POR 2 1/4,5.0,24450,.CREDITO,LABORATORIO ELEA PHOENIX S A
2,2023-03-30 09:24:52,BEBIDA GASEOSA POR 2 1/4,2.0,13300,EFECTIVO,AGUAS BONAERENSES S.A.
3,2023-03-29 16:12:46,ALMUERZO,2600.0,2600,.CREDITO,LABORATORIO ELEA PHOENIX S A
4,2023-03-29 16:10:34,SERVICIO DE CATERING DEL DIA 5 DESEPTIEMBRE,10500.0,10500,.CREDITO,ROEMMERS S A I C F


ELIMINO EL EFECTO INFLACIONARIO


In [ ]:
import pandas as pd

# 1. Leé tus ventas
df = pd.read_csv(MOSTRADOR_SALIDA_PATH, parse_dates=['Fecha'])

# 2. Filtrá solo enero-junio de 2024 y 2025
df['Anio'] = df['Fecha'].dt.year
df['Mes'] = df['Fecha'].dt.month

# 3. Armá la tabla de inflación acumulada

# Inflación mensual en porcentaje
inflaciones_pct = [
    20.6, 13.2, 11.0, 8.8, 4.2, 4.6, 4.0, 4.2, 3.5, 2.7, 2.4, 2.7,  # 2024 (ene-dic)
    2.2, 2.4, 3.7, 2.8, 1.5, 1.6, 1.9, 1.5                                   # 2025 (Agregar las inlfaciones mensuales)
]

# Convertir a decimal
inflaciones = [x / 100 for x in inflaciones_pct]

# Calcular coeficientes acumulados a agosto 2025
coef_acumulados = []
for i in range(len(inflaciones)):
    factor = 1.0
    for j in range(i + 1, len(inflaciones)):
        factor *= 1 + inflaciones[j]
    coef_acumulados.append(round(factor, 3))  # redondeamos a 3 decimales

# Meses y años
anios = [2024]*12 + [2025]*8 #agregar meses si se agregan inflaciones
meses = list(range(1, 13)) + list(range(1, 9)) #agregar meses si se agregan inflaciones

# Crear DataFrame
inflacion = pd.DataFrame({
    'Anio': anios,
    'Mes': meses,
    'Coef_Ajuste': coef_acumulados
})

# 4. Mergea los coeficientes al dataset de ventas
df = df.merge(inflacion, on=['Anio','Mes'], how='left')

# 5. Ajustá el monto de cada venta a valores de junio 2025
df['TotalVenta_Ajustado'] = df['TotalVenta'] * df['Coef_Ajuste']

# Redondeá y convertí a entero
df['TotalVenta_Ajustado'] = df['TotalVenta_Ajustado'].round(0).astype('Int64')

# 6. Guardá el dataset ajustado
df.to_csv(MOSTRADOR_SALIDA_PATH, index=False, encoding="utf-8-sig")
print("¡Dataset ajustado guardado como ventas_mostrador.csv!")


¡Dataset ajustado guardado como ventas_mostrador.csv!


LIMPIO FACTURACION

In [3]:
import pandas as pd

# 1. Leé tus ventas
df = pd.read_csv(FACTURACION_SALIDA_PATH, parse_dates=['fecha_emision'])

# Ajustar el importe total para notas de crédito (tipo 3 y 8)
df.loc[df['tipo_comprobante'].isin([3, 8]), 'importe_total'] *= -1

# 2. Filtrá solo enero-junio de 2024 y 2025
df['Anio'] = df['fecha_emision'].dt.year
df['Mes'] = df['fecha_emision'].dt.month

# 3. Armá la tabla de inflación acumulada

# Inflación mensual en porcentaje
inflaciones_pct = [
    20.6, 13.2, 11.0, 8.8, 4.2, 4.6, 4.0, 4.2, 3.5, 2.7, 2.4, 2.7,  # 2024 (ene-dic)
    2.2, 2.4, 3.7, 2.8, 1.5, 1.6, 1.9, 1.5                                   # 2025 (ene-agosto)
]

# Convertir a decimal
inflaciones = [x / 100 for x in inflaciones_pct]

# Calcular coeficientes acumulados a agosto 2025
coef_acumulados = []
for i in range(len(inflaciones)):
    factor = 1.0
    for j in range(i + 1, len(inflaciones)):
        factor *= 1 + inflaciones[j]
    coef_acumulados.append(round(factor, 3))  # redondeamos a 3 decimales

# Meses y años
anios = [2024]*12 + [2025]*8 #agregar meses si se agregan inflaciones
meses = list(range(1, 13)) + list(range(1, 9)) #agregar meses si se agregan inflaciones

# Crear DataFrame
inflacion = pd.DataFrame({
    'Anio': anios,
    'Mes': meses,
    'Coef_Ajuste': coef_acumulados
})

# 4. Mergea los coeficientes al dataset de ventas
df = df.merge(inflacion, on=['Anio','Mes'], how='left')

# 5. Ajustá el monto de cada venta a valores de junio 2025
df['TotalVenta_Ajustado'] = df['importe_total'] * df['Coef_Ajuste']

# Redondeá y convertí a entero
df['TotalVenta_Ajustado'] = df['TotalVenta_Ajustado'].round(0).astype('Int64')



# 6. Guardá el dataset ajustado
df.to_csv(FACTURACION_SALIDA_PATH, index=False, encoding="utf-8-sig")
print("¡Dataset ajustado guardado como ventas_facturacion.csv!")

¡Dataset ajustado guardado como ventas_facturacion.csv!
